In [1]:
%load_ext autoreload
# %reload_ext autoreload

# Reload all modules imported with %aimport every time before executing the Python code typed
%autoreload 1

%aimport context_nn 
%aimport phrase_feeder 
%aimport bitnotes
%aimport watch_point
%aimport cluster
%aimport constants
import numpy as np
from bitarray import bitarray
from context_nn import ContextNN
from watch_point import WatchPoint
from cluster import Cluster
from phrase_feeder import PhraseFeeder
from bitnotes import BitNotes
from pprint import pprint
import math
import constants as const

In [2]:
import pickle

def load_cxnn(file_name: str) -> ContextNN:
    with open(file_name, 'rb') as f:
        cxnn = pickle.load(f)
        return cxnn
    
def load_noted_phrases(file_name: str) -> list:
    with open(file_name, 'rb') as f:
        noted_phrases = pickle.load(f)
        return noted_phrases

In [38]:
cxnn = load_cxnn('./data/cxnn_langs.pickle')
noted_phrases = load_noted_phrases('./data/texts/tst_epo.pickle')

In [39]:
%%time

cxnn.start_detection()

for num, phrase in enumerate(noted_phrases):
    bit_chord = cxnn.bit_notes.phrase_chord(phrase)
    cxnn.detect_bits(bit_chord)
    if num % 500 == 0:
        print(f'phrase # {num}/{len(noted_phrases)}')

phrase # 0/3210
phrase # 500/3210
phrase # 1000/3210
phrase # 1500/3210
phrase # 2000/3210
phrase # 2500/3210
phrase # 3000/3210
Wall time: 1min 16s


In [ ]:
print(cxnn.cluster_count())
pprint(cxnn.cluster_len_stats())
pprint(cxnn.cluster_activity_stats())

In [40]:
cxnn.summarize_detection(point_threshold=500)

0 165.05099076495827
1 172.43316157209185
2 142.39440672499816
3 193.0204902438627
4 200.74654692866446
5 244.79383865812818
6 219.06532161437173
7 200.8484781931472


[0, 0, 0, 0, 0, 0, 0, 0]

In [10]:
cxnn.data_marks

{(0,): 'bel',
 (1,): 'blg',
 (2,): 'eng',
 (3,): 'epo',
 (4,): 'jbo',
 (5,): 'pol',
 (6,): 'rus',
 (7,): 'ukr'}

In [43]:
cxnn.vectors_received

4330

In [15]:
print(cxnn.cluster_count())
print(cxnn.point_stats())

23445
[(2, 0), (4, 5), (5, 1), (2, 12), (5, 0), (3, 28), (3, 15), (5, 0), (5, 0), (4, 0), (2, 0), (6, 0), (3, 127), (1, 0), (7, 0), (4, 22), (7, 11), (6, 10), (7, 12), (1, 71), (0, 62), (2, 0), (0, 0), (0, 6), (1, 1), (5, 6), (4, 1), (6, 23), (0, 167), (0, 166), (1, 17), (7, 140), (6, 0), (6, 1), (7, 20), (2, 0), (4, 0), (4, 80), (3, 27), (0, 4), (3, 42), (5, 0), (1, 1), (0, 49), (2, 0), (6, 0), (0, 0), (1, 1), (2, 0), (2, 0), (5, 0), (4, 55), (6, 25), (5, 0), (1, 1), (3, 167), (4, 1), (1, 60), (1, 97), (7, 0), (6, 3), (7, 62), (7, 34), (3, 79), (6, 0), (5, 0), (4, 2), (5, 41), (2, 0), (7, 79), (4, 13), (3, 5), (7, 206), (2, 0), (2, 1), (0, 46), (6, 0), (0, 222), (0, 69), (3, 12), (6, 0), (1, 0), (7, 109), (6, 20), (5, 16), (4, 107), (6, 12), (1, 0), (4, 7), (3, 54), (0, 6), (3, 49), (5, 0), (3, 40), (3, 118), (7, 21), (3, 28), (7, 100), (3, 146), (2, 0), (2, 0), (5, 0), (3, 6), (1, 6), (4, 69), (3, 39), (5, 11), (2, 0), (7, 60), (4, 12), (0, 36), (5, 0), (4, 37), (5, 4), (5, 11), (4, 

In [11]:
sum([624, 342637, 0.0, 351069, 2021, 789, 350828, 2955, 274, 1810, 337907, 1010, 3099])

1395023.0

In [13]:
sum([556, 593, 1233, 3074, 4708, 689398, 24, 10971, 4258, 492475, 2502, 6425])

1216217

In [ ]:
[0.0, 0.0, 596, 268, 520, 81307, 0.0, 0.0, 0.0, 0.0, 310, 0.0, 79]
[0.0, 0.0, 678, 189, 268, 111389, 0.0, 0.0, 0.0, 0.0, 158, 0.0, 50]